<a href="https://colab.research.google.com/github/suphawadeeth/Building-Generative-AI-Applications-with-Gradio/blob/main/05_LLM_chatbot_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chat with any LLM!** 💬

In this project, we are building a chatbot app with an open-source LLM **Falcon40B**.

We'll be using an Inference Endpoint for **Falcon-40B-Instruct** , one of best ranking open source LLM on the 🤗 Hugging Face Open LLM Leaderboard.



---
Load your HF API key and relevant Python libraries


In this project, ee are building a chat app with an open-source LLM.

In [ ]:
import os
import io
import IPython.display
from PIL import Image
import base64
import requests
requests.adapters.DEFAULT_TIMEOUT = 60
import requests, json
from text_generation import Client
from google.colab import userdata
hf_api_key = userdata.get('HF_API_KEY')

#FalcomLM-instruct endpoint on the text_generation library
client = Client(userdata.get('HF_API_FALCOM_BASE'), headers={"Authorization": f"Basic {hf_api_key}"}, timeout=120)

## **Building an app to chat with any LLM**



In [ ]:
prompt = "Has math been invented or discovered?"
client.generate(prompt, max_new_tokens=256).generated_text

In [ ]:
import gradio as gr
def generate(input, slider):
    output = client.generate(input, max_new_tokens=slider).generated_text
    return output

demo = gr.Interface(fn=generate,
                    inputs=[gr.Textbox(label="Prompt"),
                            gr.Slider(label="Max new tokens",
                                      value=20,
                                      maximum=1024,
                                      minimum=1)],
                    outputs=[gr.Textbox(label="Completion")])

gr.close_all()
demo.launch(share=True, server_port=int(os.environ['PORT1']))

Since the model does not have a memory. We could not lead a conversation.

Therefore, to create a chatbot, we need to send a context of the question/answer and the follow up question to a model.

Gradio can help with this task as a streamline conversation by sending the history conversation to the model.

---

**gr.Chatbot()**

- `gr.Chatbot()` allows you to save the chat history (between the user and the LLM) as well as display the dialogue in the app.

- Define your `function` to take in a `gr.Chatbot()` object.

  - Within your defined fn function, append a tuple (or a list) containing the user message and the LLM's response:
      
      `chatbot_object.append( (user_message, llm_message) )`
- Include the chatbot object in both the inputs and the outputs of the app.

In [ ]:
import random

def respond(message, chat_history):
        #No LLM here, just respond with a random pre-made message
        bot_message = random.choice(["Tell me more about it",
                                     "Cool, but I'm not interested",
                                     "Hmmmm, ok then"])
        chat_history.append((message, bot_message))
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit

gr.close_all()
demo.launch()

**Format the prompt with the chat history¶**

- You can iterate through the chatbot object with a for loop.
- Each item is a tuple containing the user message and the LLM's message.




```
for turn in chat_history:
    user_msg, bot_msg = turn
    ...
```




In [ ]:
def format_chat_prompt(message, chat_history):
    """This function creates a history of the question-response and return them as a prompt
    to feed the API"""
    prompt = ""
    for turn in chat_history:
        user_message, bot_message = turn
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    return prompt

def respond(message, chat_history):
    """The function feed the model (API) using prompt (chat hisory) from the previous conversation"""
        formatted_prompt = format_chat_prompt(message, chat_history) # we past formatted_prompt (the history conversation) to the API
        bot_message = client.generate(formatted_prompt,
                                     max_new_tokens=1024,
                                     stop_sequences=["\nUser:", "<|endoftext|>"]).generated_text #to prevent the bot from impersonate the user message
        chat_history.append((message, bot_message))
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit

gr.close_all()
demo.launch(share=True, server_port=int(os.environ['PORT3']))

## **Adding other advanced features**

## **Streaming**

If your LLM can provide its tokens one at a time in a stream, you can accumulate those tokens in the chatbot object.

The for loop in the following function goes through all the tokens that are in the stream and appends them to the most recent conversational turn in the chatbot's message history.

In [ ]:
def format_chat_prompt(message, chat_history, instruction):
    prompt = f"System:{instruction}" #we add the system intruction here to create a persona for the bot
    for turn in chat_history:
        user_message, bot_message = turn
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    return prompt

def respond(message, chat_history, instruction, temperature=0.7):
    prompt = format_chat_prompt(message, chat_history, instruction)
    chat_history = chat_history + [[message, ""]]
    stream = client.generate_stream(prompt,
                                      max_new_tokens=1024,
                                      stop_sequences=["\nUser:", "<|endoftext|>"],
                                      temperature=temperature)
                                      #stop_sequences to not generate the user answer
    acc_text = ""
    #Streaming the tokens
    for idx, response in enumerate(stream):
            text_token = response.token.text

            if response.details:
                return

            if idx == 0 and text_token.startswith(" "):
                text_token = text_token[1:]

            acc_text += text_token
            last_turn = list(chat_history.pop(-1))
            last_turn[-1] += acc_text
            chat_history = chat_history + [last_turn]
            yield "", chat_history
            acc_text = ""

In [ ]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    with gr.Accordion(label="Advanced options",open=False):
        system = gr.Textbox(label="System message", lines=2, value="A conversation between a user and an LLM-based AI assistant. The assistant gives helpful and honest answers.")
        temperature = gr.Slider(label="temperature", minimum=0.1, maximum=1, value=0.7, step=0.1) #set how much variation of the answer e.g. if set to 0, the bot will only give the same answer everytime to the same question
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot]) #Press enter to submit

gr.close_all()
demo.queue().launch())

Notice, in the cell above, you have used `demo.queue().launch()` instead of `demo.launch()`.

"queue" helps you to boost up the performance for your demo. You can read [setting up a demo for maximum performance](https://www.gradio.app/guides/setting-up-a-demo-for-maximum-performance) for more details.

In [ ]:
gr.close_all()